## **Bioprospecting analysis 2.0**

#### In this Notebook we will use the data generated in the [Bioprospecting analysis 1.0](https://github.com/pereiramemo/bioprospecting/blob/main/notebooks/bioprospecting-1.0.ipynb)
#### The tasks are organized in the following sections:  
#### **0. Set env**
#### **1. Explore the BGC class composition**
#### **2. Explore the BGC's taxonomic composition**
#### **3. Explore the BGC diversity**
#### **4. Assess the BGC novelty**

***
<a id='section0'></a>
**0. Set env**
***

Load the libraries.

In [27]:
library(tidyverse)
library(lubridate)
library(ggdendro)
library(ggpubr)
library(vegan)
options(dplyr.summarise.inform = FALSE)
source("../aux_scripts/get_season_north.R")
source("../aux_scripts/custom_rarefy.R")

Load the data.

In [28]:
BGC_TBL <- read_tsv("./tables/bgc_abund.tsv.gz", 
                    col_names = F,
                    show_col_types = F)
colnames(BGC_TBL) <- c("sample", "seq_id", "edge", "bgc_class", "abund")

TAXA_TBL <- read_tsv("./tables/bgc_taxa_annot_lca_formatted.tsv.gz", 
                     col_names = F,
                     show_col_types = F)
colnames(TAXA_TBL) <- c("sample", "seq_id", "tax_level", "tax_path")

METADATA_TBL <- read_csv("./tables/SOLA_metadata_extended.tsv.gz",
                         col_names = T,
                         show_col_types = F)
METADATA_TBL$Season <- factor(get_season_north(METADATA_TBL$collection_date), 
                             levels = c("Winter", "Spring", "Summer", "Fall"))

***
**1. Explore the BGC class composition**
***

First we have to collapse the BGC class abundance per sample.

In [29]:
BGC_TBL_coll <- BGC_TBL %>%
                group_by(sample, bgc_class) %>%
                summarize(abund = sum(abund)) %>%
                group_by(sample) %>%
                mutate(abund_rel = 100*abund/sum(abund)) %>%
                left_join(x =., 
                          y = METADATA_TBL %>% select(run_accession, collection_date),
                           by = c("sample" = "run_accession")) %>%
                ungroup %>%
                select(-sample) %>%
                mutate(Season = get_season_north(collection_date)) %>%
                mutate(collection_date = as.character(collection_date))

head(BGC_TBL_coll)

bgc_class,abund,abund_rel,collection_date,Season
<chr>,<dbl>,<dbl>,<chr>,<fct>
betalactone,1.014010,14.109479,2012-01-03,Winter
phosphonate,1.041835,14.496650,2012-01-03,Winter
resorcinol,1.011193,14.070280,2012-01-03,Winter
terpene,4.119691,57.323591,2012-01-03,Winter
betalactone,3.185653,23.041487,2012-02-21,Winter
hserlactone,1.068975,7.731779,2012-02-21,Winter


Let's get some basic stats:

In [30]:
bgc_class_n <- BGC_TBL_coll$bgc_class %>% unique %>% length
bgc_seqs_n <- BGC_TBL$seq_id %>% unique %>% length
bgc_complete_seqs_n <- sum(!BGC_TBL$edge)

cat(paste(rep("#",80), sep = "", collapse = ""), "\n")
cat(paste("Number of BGC classes:", bgc_class_n, sep = " "),"\n")
cat(paste("Number of BGC sequences:", bgc_seqs_n, sep = " "),"\n")
cat(paste("Number of complete BGC sequences:", bgc_complete_seqs_n, sep = " "),"\n")
cat(paste(rep("#",80), sep = "", collapse = ""), "\n")

################################################################################ 
Number of BGC classes: 16 
Number of BGC sequences: 744 
Number of complete BGC sequences: 75 
################################################################################ 


And count the number of complete BGC sequences per class:

In [31]:
bgc_complete_seqs_counts <- BGC_TBL %>% filter(edge == F) %>% select(bgc_class) %>% table()
as.data.frame(bgc_complete_seqs_counts)

bgc_class,Freq
<fct>,<int>
arylpolyene,1
betalactone,5
hserlactone,3
phosphonate,1
RiPP-like,7
siderophore,1
T1PKS,1
terpene,56


***

To get a general overview of the BGC class composition, let's create a barplot of relative abundances.

In [ ]:
bgc_colors <- c('#8c510a','#bf812d','#dfc27d','#f6e8c3',
                '#c7eae5','#80cdc1','#35978f','#01665e',
                '#abdda4','#66c2a5', '#d53e4f','#f46d43',
                '#fdae61','#40004b','#762a83', '#9970ab')

season_colors <- c("#154360", "#3c7810", "#c93f1b","#98482b")

bgc_class_names <- unique(BGC_TBL_coll$bgc_class)
names(bgc_colors) <- bgc_class_names[order(bgc_class_names)]

text_size <- 12
BGC_TBL_coll_div <- BGC_TBL_coll %>%
                    group_by(collection_date) %>%
                    summarize(shannon = diversity(abund_rel))

barplot <- ggplot(BGC_TBL_coll, aes(x = collection_date, y = abund_rel, fill = bgc_class)) + 
           geom_bar(stat = "identity", width = 0.5, alpha = 0.7) +
           xlab("Sample") +
           ylab("Percentage of abundance") +
           scale_fill_manual(name="BGC class",
                             values = bgc_colors) +
           scale_y_continuous(expand=c(0,0.1), limits = c(-1,101), 
                              sec.axis = sec_axis(~.*(max(BGC_TBL_coll_div$shannon)/101), name = "Shannon diversity")) +
           theme_light() +
           theme( 
                 axis.text.x = element_text(size = text_size -2, angle = 45, 
                                            hjust = 1, color = "black"),
                 axis.text.y = element_text(size = text_size, color = "black"),
                 axis.title.x = element_text(size = text_size + 4, color = "black",
                                             margin = margin(10,0,0,0)),
                 axis.title.y = element_text(size = text_size + 4, color = "black",
                                             margin = margin(0,15,0,0)),
                 legend.text = element_text(size = text_size +2, color = "black"),
                 legend.title = element_text(size = text_size +4 , color = "black"),
                 legend.position = "right",
                 legend.margin = ggplot2::margin(0,0,-5,0),
                 strip.background = element_blank(),
                 strip.text = element_text(color = "black", size = text_size)) +
                 guides(fill = guide_legend(keywidth = 0.6, keyheight = 0.6)) +
          geom_line(data = BGC_TBL_coll_div, aes(y = shannon*(99/1.5), fill = NULL, group = 1),
                     linewidth = 1.5) 

options(repr.plot.width=15, repr.plot.height=8)
barplot

**Figure 1**. Barplot representing the relative abundance of BGC classes per sample (left Y axis). The black curve represents the BGC class Shannon diversity (right Y axis). To compute the abundance of each BGC class within a sample, we summed the coverage of all the BGC sequences of the same class.

**Main observations**: As expected for surface marine environments, we observed a high abundance of terpenes. Similar results have been obtained by [Paoli 2022](https://www.nature.com/articles/s41586-022-04862-3) and [Pereira 2020](https://www.biorxiv.org/content/10.1101/2021.01.20.427441v1). This finding could be explained by a higher abundance of carotenoid pigments involved in light harvesting and oxidative stress protection processes.

***

To gain more insights about this data, let's create a two-way clustering heatmap.  
First, we generate the clusters:

In [ ]:
BGC_TBL_coll_wide <- BGC_TBL_coll %>%
                     select(-abund_rel, -Season) %>%
                     pivot_wider(names_from = bgc_class, 
                                 values_from = abund, 
                                 values_fill = 0) %>%
                     column_to_rownames("collection_date")

BGC_TBL_norm <- decostand(BGC_TBL_coll_wide, method = "hellinger")
sample_clust <- hclust(vegdist(BGC_TBL_norm, method = "euclidean"), method="ward.D")
bgc_clust <- hclust(vegdist(t(BGC_TBL_norm), method = "euclidean"), method="ward.D")

sample_clust_dendro <- sample_clust %>%
                       as.dendrogram
bgc_clust_dendro <- bgc_clust %>%
                    as.dendrogram 

sample_clust_data <- dendro_data(sample_clust_dendro)
bgc_clust_data <- dendro_data(bgc_clust_dendro)


And the dendro plots.

In [ ]:
sample_clust_metadata <- data.frame(collection_date = label(sample_clust_data)$label %>% 
                                    as.character,
                                    x = label(sample_clust_data)$x,
                                    y = label(sample_clust_data)$y) %>%
                         mutate(Season = get_season_north(collection_date))

season_colors <- c("#154360", "#3c7810", "#c93f1b","#98482b")

sample_clust_plot <- ggplot(segment(sample_clust_data)) +
                     coord_flip() +
                     geom_segment(aes(x=x, y=y, xend=xend, yend=yend), linewidth = 0.2) +
                     geom_point(data = sample_clust_metadata, 
                                aes(x = x , y = y, colour = Season),
                                    shape = 15, size = 4, alpha = 0.8) +
                     scale_colour_manual(values=season_colors) +
                     theme_void() +
                     theme(plot.margin = unit(c(-1,1,6.5,0.2), "lines"))

bgc_clust_plot <- ggplot(segment(bgc_clust_data)) +
                  geom_segment(aes(x=x, y=y, xend=xend, yend=yend), linewidth = 0.2) +
                  theme_void() +
                  theme(plot.margin = unit(c(1,14.5,-0.5,7.5), "lines"))

Now the heatmap.

In [ ]:
BGC_TBL_norm_long <- BGC_TBL_norm %>%
                     as.data.frame %>%
                     rownames_to_column("collection_date") %>%
                     pivot_longer(names_to = "bgc_class", 
                                  values_to = "abund", 
                                  cols = 2:(dim(BGC_TBL_norm)[2]+1))
                     
BGC_TBL_norm_long$collection_date <- factor(BGC_TBL_norm_long$collection_date, 
                                            levels = sample_clust$labels[sample_clust$order])
BGC_TBL_norm_long$bgc_class <- factor(BGC_TBL_norm_long$bgc_class, 
                                      levels = bgc_clust$labels[bgc_clust$order])
heatmap <- ggplot(BGC_TBL_norm_long, aes(x = collection_date, y = bgc_class, fill = abund)) + 
           geom_tile() +
           xlab("Collection date") +
           ylab("BGC class") +
           theme_light() +
           theme( 
                 axis.text.x = element_text(size = text_size, angle = 45, hjust = 1, color = "black"),
                 axis.text.y = element_text(size = text_size, color = "black"),
                 axis.title.x = element_text(size = text_size + 4, color = "black",
                                             margin = margin(10,0,0,0)),
                 axis.title.y = element_text(size = text_size + 4, color = "black",
                                             margin = margin(0,15,0,0)),
                 legend.text = element_text(size = text_size +2, color = "black"),
                 legend.title = element_text(size = text_size +4 , color = "black"),
                 legend.position = c(1.3,0.95),
                 legend.margin = ggplot2::margin(0,0,-5,0),
                 strip.background = element_blank(),
                 strip.text = element_text(color = "black", size = text_size),
                 plot.margin = unit(c(0.2,0,1,1), "lines")) +
            guides(fill = guide_legend(keywidth = 0.6, keyheight = 0.6)) +
            scale_fill_gradient(low="white", high="#01665e", name = "% of abundance") +
            coord_flip()

Lastly, merge all plots.

In [ ]:
heatmap_1 <- ggarrange(heatmap, sample_clust_plot,
                       nrow = 1, ncol = 2, 
                       widths = c(0.7, 0.3))

heatmap_1and2 <- ggarrange(bgc_clust_plot, heatmap_1,    
                           nrow = 2, ncol = 1,  
                           heights = c(0.2, 0.8))

options(repr.plot.width=10, repr.plot.height=12)
heatmap_1and2

**Figure 2.** Two-way clustering heatmap of the metagenomic samples based on their BGC class composition. For this analysis, the BGC class abundance matrix was normalized utilizing the Hellinger transformation. To cluster the rows and columns we used the Ward method and the euclidean distance.  

**Main observations**: The two main clusters of the samples separate the semesters winter and fall from summer and spring. It is also noticeable that the later semester has a greater abundance of terpenes.

***
**2. Explore the BGCs’ taxonomic composition**
***

Here we are going to analyze the distribution of the number of BGC sequences per class and phyla.  
First we need to match the BGC abundance and taxonomic annotation tables and get the phylum annotations.

In [ ]:
bgc2taxa <- left_join(x = BGC_TBL, y = TAXA_TBL, by = c("seq_id","sample")) %>%
            mutate(tax_path2 = if_else(grepl(pattern = ".*;p_[^;]+;.*", tax_path, perl = T), tax_path, "p_Unclassified;")) %>%
            mutate(phylum = sub(".*p_([^;]+);.*", "\\1", tax_path2)) %>%
            select(-tax_path, -tax_level)

head(bgc2taxa, n = 5)

Now that we have all organized in a single table, we can create the plots.

In [ ]:
phyla_names <- unique(bgc2taxa$phylum)
i <- phyla_names!="Unclassified"
phyla_names <- c(phyla_names[i][order(phyla_names[i])], "Unclassified")

bgc2taxa_counts <- bgc2taxa %>%
                   group_by(phylum, bgc_class) %>%
                   summarize(bgc_seqs_n = unique(seq_id) %>% length) %>%
                   ungroup()

bgc2taxa_counts$phylum <- factor(bgc2taxa_counts$phylum, levels = phyla_names)

phyla_barplot <- ggplot(data = bgc2taxa_counts, aes(x = bgc_class, y = bgc_seqs_n +1)) + 
                 facet_wrap(~phylum, scales = "free_y", ncol = 5) +
                 geom_bar(stat = "identity", width = 0.5, alpha = 0.7) +
                 xlab("BGC class") +
                 ylab("log10 of number of BGC sequences") +
                 scale_y_log10() +
                 theme_light() +
                 theme( 
                       axis.text.x = element_text(size = text_size -2, angle = 45, hjust = 1, color = "black"),
                       axis.text.y = element_text(size = text_size, color = "black"),
                       axis.title.x = element_text(size = text_size + 4, color = "black",
                                                   margin = margin(10,0,0,0)),
                       axis.title.y = element_text(size = text_size + 4, color = "black",
                                                   margin = margin(0,15,0,0)),
                       legend.text = element_text(size = text_size +2, color = "black"),
                       legend.title = element_text(size = text_size +4 , color = "black"),
                       legend.position = "right",
                       legend.margin = ggplot2::margin(0,0,-5,0),
                       strip.background = element_blank(),
                        strip.text = element_text(color = "black", size = text_size)) +
                 guides(fill = guide_legend(keywidth = 0.6, keyheight = 0.6)) 

options(repr.plot.width=24, repr.plot.height=14)
phyla_barplot


**Figure 3.** For each phyla, barplots representing the number of BGC sequences (log10) of the different classes. Unclassified refers to all BGC sequences that did not have a taxonomic annotation at the phylum level. 

**Main observations**: The Pseudomonadota (also named Proteobacteria), is the phylum with the largest number of BGC sequences. Notably, we observed a large number of taxonomically unclassified BGC of all 16 BGC classes.

***

Now let's analyze the taxonomic composition of the complete BGC sequences.

In [ ]:
phyla_colors <-  as.character(bgc_colors[10:16])

bgc2taxa_counts_complete <- bgc2taxa %>%
                            filter(edge == F) %>%
                            group_by(bgc_class, phylum) %>%
                            summarize(seqs_n_tax = n()) %>%
                            group_by(bgc_class) %>%
                            mutate(seqs_n_tax_rel = 100*seqs_n_tax/sum(seqs_n_tax))

phyla_names <- unique(bgc2taxa_counts_complete$phylum)
i <- phyla_names!="Unclassified"
phyla_names <- c(phyla_names[i][order(phyla_names[i])], "Unclassified")
bgc2taxa_counts_complete$phylum <- factor(bgc2taxa_counts_complete$phylum, levels = phyla_names)

taxa2bgc_barplot <- ggplot(bgc2taxa_counts_complete, aes(x = bgc_class, y = seqs_n_tax, fill = phylum)) + 
                    geom_bar(stat = "identity", width = 0.5, alpha = 0.7) +
                    xlab("BGC class") +
                    ylab("Number of BGC sequences") +
                    scale_fill_manual(name="Phylum",
                                      values = phyla_colors) +
                  #  scale_y_continuous(expand=c(0,0.1), limits = c(-1,101)) +
                    theme_light() +
                    theme( 
                          axis.text.x = element_text(size = text_size +1, angle = 45, hjust = 1, color = "black"),
                          axis.text.y = element_text(size = text_size, color = "black"),
                          axis.title.x = element_text(size = text_size + 4, color = "black",
                                                      margin = margin(10,0,0,0)),
                          axis.title.y = element_text(size = text_size + 4, color = "black",
                                                      margin = margin(0,15,0,0)),
                          legend.text = element_text(size = text_size +2, color = "black"),
                          legend.title = element_text(size = text_size +4 , color = "black"),
                          legend.position = "right",
                          legend.margin = ggplot2::margin(0,0,-5,0),
                          strip.background = element_blank(),
                          strip.text = element_text(color = "black", size = text_size)) +
                    guides(fill = guide_legend(keywidth = 0.6, keyheight = 0.6)) 

options(repr.plot.width=8, repr.plot.height=6)
taxa2bgc_barplot

**Figure 4**. Barplots representing the number of complete BGC sequences per class. Different colors in each bar represent different phyla. Complete BGC sequences are identified as those that are not annotated on an edge of a contig.

**Main observations**: The Unclassified group concentrates the majority of the BGC sequences, while the terpenes appear as the BGC class with the highest number of complete BGC sequences, probably, due to its higher abundance and relatively shorter length.  

***
**3. Explore the BGC diversity**
***

In this analysis we are going to create rarefaction curves based on the Gene Cluster Families (GCFs) obtained with [BiG-SCAPE](https://github.com/medema-group/BiG-SCAPE).  
For this, we will follow the same [approach](https://en.wikipedia.org/wiki/Rarefaction_(ecology)) that is traditionally applied when working with Operation Taxonomic Units (OTUs).

Let's first load the clustering data.

In [ ]:
BGC_CLUST <- read_tsv("./tables/all_clustering_c1.00.tsv.gz", 
                     col_names = T, show_col_type = F)
head(BGC_CLUST, n = 5)

Then we construct a sample x GCF abundance matrix.  
Thus, we have to map the BGCs' abundance data from our `BGC_TBL` table.

In [ ]:
bcg_clust2abund <- BGC_CLUST %>%
                   mutate(seq_id = sub(x = seq_id, pattern = ".region.*", replacement = "")) %>%
                   left_join(x = .,
                             y = BGC_TBL %>% select(-sample),
                             by = "seq_id") %>%
                   select(bgc, sample, cluster_id, abund) %>%
                   group_by(bgc, sample, cluster_id) %>%
                   summarize(abund = sum(abund)) %>%
                   left_join(x =., 
                             y = METADATA_TBL %>% select(run_accession, collection_date),
                             by = c("sample" = "run_accession")) %>%
                    ungroup %>%
                    select(-sample) %>%
                    mutate(sample = as.character(collection_date))
               

Next, we rarefy the data using our custom R function: [custom_rarefy](https://github.com/pereiramemo/bioprospecting/blob/main/aux_scripts/custom_rarefy.R) and create the plot of the rarefaction curves.

In [ ]:
bgc_names <- unique(bcg_clust2abund$bgc)
rare_plots_list <- list()

for (i in bgc_names) {
  
  rare_data <- custom_rarefy(TBL = bcg_clust2abund, bgc_name = i, n_cores = 20)
  
  if (is.null(rare_data)) { 
    
    next()
    
  }
  
  rare_data$rare$Season <- get_season_north(rare_data$rare$sample)
  rare_plots_list[[i]] <- ggplot(rare_data$rare, 
                                 aes(x = iteration, 
                                     y = richness, 
                                     group = sample, 
                                     color = Season)) +
                          scale_color_manual("Season", values = season_colors) +
                          geom_line(linewidth = 1.2, alpha = 0.7) +
                          ylab("Richness") +
                          xlab("Sample size") +
                          scale_x_continuous(labels = scales::scientific, 
                                             limits = c(0,  max(rare_data$rare$iteration) + 3)) +
                          scale_y_continuous(labels = scales::scientific) +
                          theme_bw() +
                          ggtitle(i) +
                          theme( 
                                axis.text.x = element_text(size = text_size +1, color = "black"),
                                axis.text.y = element_text(size = text_size, color = "black"),
                                axis.title.x = element_text(size = text_size + 4, color = "black",
                                                            margin = margin(10,0,0,0)),
                                axis.title.y = element_text(size = text_size + 4, color = "black",
                                                            margin = margin(0,15,0,0)),
                                legend.text = element_text(size = text_size +2, color = "black"),
                                legend.title = element_text(size = text_size +4 , color = "black"),
                                legend.position = "right",
                                legend.margin = ggplot2::margin(0,0,-5,0),
                                strip.background = element_blank(),
                                strip.text = element_text(color = "black", size = text_size),
                                plot.title = element_text(size = text_size + 8, face = "bold", hjust = 0.5)) +
                           guides(fill = guide_legend(keywidth = 0.6, keyheight = 0.6)) +
                          annotate("text", 
                                   label = rare_data$coords$sample, 
                                   x = rare_data$coords$max_interation, 
                                   y = rare_data$coords$max_richness+ 1,
                                   size = 4, check_overlap = T) 
}


The only BGC class for which we can create this plot are the **Terpenes**, for all the others we don't have enough data.

In [ ]:
options(repr.plot.width=16, repr.plot.height=8)
rare_plots_list$Terpene

**Figure 5**. Rarefaction curves of the Gene Cluster Families (GCFs) of Terpenes. To create these curves we used the Terpene GCF abundance matrix, which comprised 85 different GCFs.   

**Main observations:** This figure shows that the samples from Summer and Spring have a much higher abundance of terpenes and a greater diversity. It is worth noting that none of the curves reach a plateau, that is, a greater sampling effort is needed to capture the diversity of terpenes in these environments. 

***
<a id='section4'></a>
**4. Assess the BGC novelty**
***

Lastly, let's assess the novelty of all BGC sequences utilizing [MIBiG DB](https://mibig.secondarymetabolites.org/) as a reference.  
For this, we are going to use the output of [BiG-SCAPE](https://github.com/medema-group/BiG-SCAPE), where we clustered the BGC sequences identified in our metagenomic samples together with BGC sequences of MIBiG.  
Let's load the table with all the distances between query and reference BGC sequences.

In [ ]:
BGC_DIST <- read_tsv("./tables/all_c1.00.network.gz", 
                     col_names = T, show_col_type = F)

head(BGC_DIST)

And now extract our metagenomic BGC sequences.

In [ ]:
BGC_DIST_redu <- BGC_DIST %>% 
                 filter( grepl(x = `Clustername 1`, pattern = "BGC", perl = T) & grepl(x = `Clustername 2`, pattern = "ERR", perl = T) ) 
head(BGC_DIST_redu)

Let's select the BGCs from MIBiG that have the shortest distance to our metagenomic BGCs.

In [ ]:
BGC_DIST_redu_min_dist <- BGC_DIST_redu %>%
                          mutate(`Clustername 2` = sub(x = `Clustername 2`, pattern = ".region.*", replacement = "")) %>%
                          left_join(x = ., 
                                    y = bgc2taxa, 
                                    by = c("Clustername 2" = "seq_id")) %>%
                          group_by(bgc, `Clustername 2`) %>%
                          mutate(min_dist = min(`Raw distance`)) %>%
                          filter(`Raw distance` <= min_dist)
                          

head(BGC_DIST_redu_min_dist, n = 5)

Let's see this in a boxplot.

In [ ]:
phyla_names <- as.character(unique(BGC_DIST_redu_min_dist$phylum))
i <- phyla_names!="Unclassified"
phyla_names <- c(phyla_names[i][order(phyla_names[i])], "Unclassified")
BGC_DIST_redu_min_dist$phylum <- factor(BGC_DIST_redu_min_dist$phylum, levels = phyla_names)

phyla_colors <- as.character(bgc_colors[1:length(phyla_names)])
names(phyla_colors) <- phyla_names

bgc_novelty_boxplot <- ggplot(BGC_DIST_redu_min_dist, aes(x = phylum, y = `Raw distance`, 
                                                          fill = phylum, color = phylum)) + 
                       facet_wrap(~bgc_class, scales = "free_x", nrow = 2) +
                       geom_boxplot(alpha = 0.7) +
                       # geom_jitter() +
                       xlab("BGC class") +
                       ylab("Minimum distance to a reference BGC") +
                       theme_light() +
                       scale_fill_manual(name="Phylum",
                                         values = phyla_colors) +
                       scale_color_manual(name="Phylum",
                                         values = phyla_colors) +
                       
                       theme( 
                             axis.text.x = element_text(size = text_size -1, angle = 45, hjust = 1, color = "black"),
                             axis.text.y = element_text(size = text_size, color = "black"),
                             axis.title.x = element_text(size = text_size + 4, color = "black",
                                                         margin = margin(10,0,0,0)),
                             axis.title.y = element_text(size = text_size + 4, color = "black",
                                                         margin = margin(0,15,0,0)),
                             strip.background = element_blank(),
                             strip.text = element_text(color = "black", size = text_size),
                             legend.text = element_text(size = text_size +2, color = "black"),
                             legend.title = element_text(size = text_size +4 , color = "black"),
                             legend.position = "none",
                             legend.margin = ggplot2::margin(0,0,-5,0)) +
                        guides(fill = guide_legend(keywidth = 0.6, keyheight = 0.6)) 

options(repr.plot.width=20, repr.plot.height=9)
bgc_novelty_boxplot

**Figure 6**. For each BGC class, boxplots representing the distribution of minimum distances between the metagenomic BGC sequence of different phyla and reference MIBiG BGC sequences.  

**Main observations:** Apart from the Siderophores, TIPKS, and the Bacteroidetes and Unclassified Arylpolyene BGC sequences, the medians of the minimum distances have a value above 0.8, indicating a high novelty with respect to MIBiG. In addition, different phyla show little variation of the minimum distance to reference BGCs, except for Arylpolyene, TIPKS, and Terpene. 

***

Let's do the same considering only the complete BGC sequences. 

In [ ]:
phyla_names <- BGC_DIST_redu_min_dist %>% filter(edge == F) %>% pull(phylum) %>% unique() %>% as.character()

bgc_novelty_boxplot <- ggplot(BGC_DIST_redu_min_dist %>% filter(edge == F), 
                              aes(x = phylum, y = `Raw distance`, fill = phylum, colour = phylum)) + 
                       facet_grid(~bgc_class, scales = "free_x") +
                       geom_boxplot(alpha = 0.7) +
                       xlab("BGC class") +
                       ylab("Distance to reference BGC") +
                       theme_light() +
                       scale_fill_manual(name="Phyla",
                                         values = phyla_colors[phyla_names[order(phyla_names)]]) +
                       scale_color_manual(name="Phyla",
                                          values = phyla_colors[phyla_names[order(phyla_names)]]) +
                       
                       theme( 
                             axis.text.x = element_text(size = text_size +1, angle = 45, hjust = 1, color = "black"),
                             axis.text.y = element_text(size = text_size, color = "black"),
                             axis.title.x = element_text(size = text_size + 4, color = "black",
                                                         margin = margin(10,0,0,0)),
                             axis.title.y = element_text(size = text_size + 4, color = "black",
                                                         margin = margin(0,15,0,0)),
                             strip.background = element_blank(),
                             strip.text = element_text(color = "black", size = text_size),
                             legend.text = element_text(size = text_size +2, color = "black"),
                             legend.title = element_text(size = text_size +4 , color = "black"),
                             legend.position = "none") +
                        guides(fill = guide_legend(keywidth = 0.6, keyheight = 0.6)) 

options(repr.plot.width=16, repr.plot.height=5)
bgc_novelty_boxplot

**Figure 7**. Same as Figure 6, but restricted to complete BGC sequences.

**Main observations:** The RiPPs complete sequences show the highest novelty with respect to MIBiG. [The RiPP BGC class is known to produce a large diversity of compounds as a result of the post translational modifications.](https://pubs.rsc.org/en/content/articlelanding/2013/np/c2np20085f), making these complete sequences an interesting case for further studies.

***

We can see which are these reference BGCs with the following command:

In [ ]:
BGC_DIST_redu_min_dist %>%
filter(edge == F && bgc_class == "RiPP-like") %>%
pull(`Clustername 1`) %>%
unique()

If we look for these BGCs in [MIBiG database](https://mibig.secondarymetabolites.org/) we can find the compounds that they produce.
Note: BGC0001546 has a new id in MIBiG which is BGC0001485.

[BGC0001546](https://mibig.secondarymetabolites.org/repository/BGC0001485/#r1c1) and [BGC0000550](https://mibig.secondarymetabolites.org/repository/BGC0000550/index.html#r1c1) are responsible for the production of chuangxinmycin (antibiotic) and salivaricin G32 (lantibiotic), respectively.